In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Load dataset
boston = fetch_openml(name="boston", version=1, as_frame=False)
X = boston.data.astype(np.float32)
y = boston.target.astype(np.float32).reshape(-1, 1)

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Standardize features (important for NN training)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [3]:
X_train

array([[ 1.2877018 , -0.50032014,  1.0332369 , ...,  0.84534293,
        -0.07433674,  1.753505  ],
       [-0.33638445, -0.50032014, -0.4131595 , ...,  1.2047412 ,
         0.43018377, -0.56147414],
       [-0.40325332,  1.0132713 , -0.71521825, ..., -0.6371765 ,
         0.06529749, -0.6515951 ],
       ...,
       [-0.40547013,  2.9593177 , -1.3033614 , ..., -0.59225154,
         0.3790101 , -0.9106925 ],
       [ 0.85189736, -0.50032014,  1.0332369 , ...,  0.84534293,
        -2.694586  ,  1.5225704 ],
       [-0.38135594, -0.50032014, -0.35216686, ...,  1.1598163 ,
        -3.1215806 , -0.25731632]], dtype=float32)

In [3]:
# convert to tensors
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

In [4]:
# Neural network model (regression)
class HousingNet(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_features, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)   # no sigmoid → regression output
        )

    def forward(self, x):
        return self.net(x)

model = HousingNet(X_train.shape[1])

In [5]:
# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [9]:
# Training loop
epochs = 1000
for epoch in range(epochs):

    # forward
    preds = model(X_train)
    loss = criterion(preds, y_train)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch:3d} | Train MSE: {loss.item():.4f}")

Epoch   0 | Train MSE: 8.9842
Epoch 100 | Train MSE: 7.9216
Epoch 200 | Train MSE: 7.0530
Epoch 300 | Train MSE: 6.3874
Epoch 400 | Train MSE: 5.8053
Epoch 500 | Train MSE: 5.2964
Epoch 600 | Train MSE: 4.8280
Epoch 700 | Train MSE: 4.3928
Epoch 800 | Train MSE: 4.0106
Epoch 900 | Train MSE: 3.6353


In [10]:
# Evaluation

with torch.no_grad():
    test_preds = model(X_test)
    test_mse = criterion(test_preds, y_test)
    rmse = torch.sqrt(test_mse)

print("\nTest RMSE:", rmse.item())


Test RMSE: 3.275773286819458
